In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from stargazer.stargazer import Stargazer
%matplotlib inline
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.api import add_constant
import json
import geopandas as gpd
import plotly.graph_objects as go
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

# Daten importieren und untersuchen

In [4]:
besucher_df = pd.read_csv("data/besucher.csv", sep = ";", decimal = ".")
kunden_df = pd.read_csv("data/kunden.csv", sep = ";", decimal = ".")
geo_df = pd.read_csv("data/geo.txt", sep = "\t")

besucher_df.dtypes
kunden_df.dtypes
geo_df.head()

,KundeNr,Niederlassung
0,K0001,Sachsen
1,K0002,NRW
2,K0003,Hessen
3,K0004,Bayern
4,K0005,Bayern


# Daten sauber machen

In [5]:
besucher_df["Geschlecht"] = besucher_df["Geschlecht"].astype("float64")
kunden_df["Zeit"] = kunden_df["Zeit"].astype("int")
besucher_df["Zeit"] = besucher_df["Zeit"].str.replace(",",".").astype("float").astype("int")
besucher_df["Einkommen"] = besucher_df["Einkommen"].str.replace(",",".")
besucher_df["Einkommen"] = besucher_df["Einkommen"].astype("float").astype("int")
kunden_df["Einkommen"] = kunden_df["Einkommen"].astype("int")
geo_df["Niederlassung"] = geo_df["Niederlassung"].replace("NRW", "Nordrhein-Westfalen")
geo_df["Niederlassung"] = geo_df["Niederlassung"].replace(["Berlin-Charlottenburg", "Berlin-Mitte", "BERLIN"], "Berlin")

In [6]:
merge_1 = pd.concat([besucher_df, kunden_df], ignore_index = True)
data_df = pd.merge(merge_1, geo_df, on = "KundeNr")
data_df

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung
0,30,39919,0.0,29,K1814,NaN,Bayern
1,41,53430,0.0,27,K1544,NaN,Baden-Württemberg
2,64,73163,0.0,32,K1945,NaN,Baden-Württemberg
3,41,36761,0.0,44,K1825,NaN,Baden-Württemberg
4,35,42684,1.0,33,K1144,NaN,Nordrhein-Westfalen
...,...,...,...,...,...,...,...
1990,70,69581,1.0,51,K0467,103751.58,Thüringen
1991,38,48301,1.0,33,K0122,64230.98,Niedersachsen
1992,67,63547,1.0,41,K1045,103123.57,Berlin
1993,37,32686,1.0,36,K1096,49215.87,Baden-Württemberg


# Outlier behandeln

In [7]:
# hier wurde bei allen ohne Eintrag beim Geschlecht eine "1" euingefügt, weil es mehr Männder als Frauen im Datensatz gibt
data_df['Geschlecht'] = data_df['Geschlecht'].fillna(1)

In [8]:
# Hier wurde das Einkommen von 3 Kunden auf den Median gesetzt und das Alter von 2 wurden auf den Durchschnitt gesetzt 
data_df.loc[data_df['KundeNr'].isin(["K0701", "K1021"]), 'Einkommen'] = int(data_df['Einkommen'].median())
data_df.loc[data_df['KundeNr'].isin(["K1083"]), 'Einkommen'] = 62840
data_df.loc[data_df['KundeNr'].isin(["K0039", "K0494"]), 'Alter'] = int(data_df['Alter'].mean())

data_df.isnull().sum()

Alter              0
Einkommen          0
Geschlecht         0
Zeit               0
KundeNr            0
Preis            891
Niederlassung      0
dtype: int64

In [9]:
result = data_df[data_df['KundeNr'] == "K1083"]
result

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung
1973,59,62840,0.0,46,K1083,92837.59,Baden-Württemberg


In [10]:
data_df.to_csv("data/df_final.csv", sep = ",", decimal = ".")